In [2]:
import tensorflow as tf
from tensorboard.plugins import projector

import os, argparse
import pickle
import cv2, spacy, numpy as np
from keras.models import model_from_json
from keras.optimizers import SGD
import joblib
from keras import backend as K
import keras
import PIL
import pandas as pd

2023-05-18 16:32:49.911338: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-18 16:32:50.011327: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-18 16:32:50.755274: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-18 16:32:58.812094: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2027] Ignoring visible gpu device (device: 1, name: Quadro P620, pci bus id: 0000:b3:00.0, compute capability: 6.1) with core count: 4. The minimum required count is 8. Yo

https://github.com/anujshah1003/Tensorboard-own-image-data-image-features-embedding-visualization

In [23]:
tf.__version__

'2.12.0'

In [34]:
PATH = os.getcwd()
LOG_DIR = PATH+ '/vis'
LOG_DIR

'/media1/data/claire/BearBackgroundsAndBehavior/vis'

In [13]:
data_path = '/media1/data/beardata/2021/2021_bodies/2021_singles_resized'
filenames = [os.path.join(data_path, os.listdir(data_path)[i]) for i in range(len(os.listdir(data_path)))]
filenames[0]

'/media1/data/beardata/2021/2021_bodies/2021_singles_resized/P1885008_0_Dune_bear.JPG'

In [16]:
images = [PIL.Image.open(filename) for filename in filenames]

image_width, image_height = images[0].size
one_square_size = int(np.ceil(np.sqrt(len(images))))
master_width = (image_width * one_square_size) 
master_height = image_height * one_square_size
spriteimage = PIL.Image.new(
    mode='RGBA',
    size=(master_width, master_height),
    color=(0,0,0,0))  # fully transparent
for count, image in enumerate(images):
    div, mod = divmod(count,one_square_size)
    h_loc = image_width*div
    w_loc = image_width*mod    
    spriteimage.paste(image,(w_loc,h_loc))
spriteimage.convert("RGB").save('sprite.jpg', transparency=0)

In [3]:
data = pd.read_csv('embeddings_2000.tsv',sep='\t', header=None)
data

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.121979,0.180500,-0.101500,0.066033,-0.135600,-0.016117,0.075402,0.191959,0.250628,-0.363889,...,0.231436,-0.046597,0.026231,0.076395,-0.139318,-0.001033,0.289854,-0.176325,-0.153972,0.133803
1,-0.045447,0.044068,-0.137763,-0.144858,-0.040309,0.061621,-0.040065,0.259091,0.143567,-0.171977,...,0.220998,-0.062991,-0.006616,0.113453,-0.187876,0.015637,0.102037,-0.032357,-0.091988,0.161042
2,-0.068580,0.066230,-0.137090,0.043328,-0.002324,0.032082,0.117614,0.227936,0.248181,-0.240752,...,0.265619,-0.063197,0.069602,0.115273,-0.135968,0.081606,0.182556,0.024831,-0.085223,-0.010674
3,-0.050450,0.048493,-0.119378,-0.118063,-0.081385,0.035440,-0.018122,0.213397,0.057327,-0.107522,...,0.114653,-0.068044,0.105045,0.160640,-0.167509,-0.019187,0.135333,0.030994,-0.154920,0.148443
4,-0.124137,-0.001892,-0.145382,-0.035002,0.048863,0.067718,-0.101065,0.163431,0.102865,-0.279686,...,0.325686,-0.051851,0.155150,0.061030,-0.220549,-0.157861,0.067562,0.039090,-0.117150,-0.052039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,-0.085494,0.026191,-0.127971,0.012615,-0.079044,0.024931,-0.046904,0.186907,0.154195,-0.259489,...,0.265945,-0.048337,0.043305,0.058428,-0.132187,0.017703,0.191423,0.005836,-0.062578,0.079440
1996,-0.016239,0.040793,-0.193723,-0.035613,-0.029744,0.056636,-0.013613,0.268369,0.094377,-0.235595,...,0.230566,-0.057523,0.089067,0.088539,-0.232052,-0.079102,0.088098,-0.074572,-0.141354,0.116078
1997,0.015663,-0.046080,-0.220073,-0.114918,-0.037029,0.003958,-0.007342,0.170786,0.205895,-0.125916,...,0.290609,-0.065544,0.029078,-0.003826,-0.183096,0.068650,0.250422,0.023708,-0.048540,0.102774
1998,-0.073056,0.049904,-0.109593,0.010521,-0.046207,-0.065451,0.099364,0.215658,0.166973,-0.192781,...,0.168179,-0.066118,0.101024,0.113179,-0.095059,0.033171,0.110148,-0.039722,-0.151745,0.043124


In [54]:
features = tf.Variable(data)
features

<tf.Variable 'Variable:0' shape=(100, 768) dtype=float64, numpy=
array([[-0.1219793 ,  0.18049994, -0.10149986, ..., -0.17632492,
        -0.15397169,  0.13380329],
       [-0.0454467 ,  0.04406803, -0.1377628 , ..., -0.0323573 ,
        -0.09198836,  0.16104163],
       [-0.06858011,  0.06622957, -0.13709007, ...,  0.02483103,
        -0.08522267, -0.01067392],
       ...,
       [-0.07277662,  0.10823404, -0.05953973, ...,  0.02689851,
        -0.15945576,  0.14109643],
       [-0.05316458,  0.13967015, -0.13529539, ...,  0.00229591,
        -0.06284451,  0.10143276],
       [-0.039361  , -0.05674601, -0.11724309, ..., -0.03087272,
        -0.2015423 ,  0.07577317]])>